# Errors
This notebook computes errors and various accuracy measures using PredictionResults() object and files generated by merge_data as inputs. This notebook computes errors and measures **for each station and for each direction separately**. The steps of the code are as follows:
- computes Mean Absolute Relative Error (MARE) and MAE to capacity ratio (ECR) depending on volume / capacity

### Hyperparameters and initialization

In [ ]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
GRAN_15_MIN = True
CHANGE_TO_ONE_HOUR = False # True if you want to change granulation from 15 minutes to 1 hour. GRAN_15_MIN must also be true


print ('You are working with {} granulated data.'
       .format('15 minute' if GRAN_15_MIN else 'hourly'))

if CHANGE_TO_ONE_HOUR and (not GRAN_15_MIN):
    CHANGE_TO_ONE_HOUR = False
    print ('15 min granulation not set. CHANGE_TO_ONE_HOUR automatically set to false.')
    
print ('Data will {}be converted from 15 min to one hour format.'
       .format('' if CHANGE_TO_ONE_HOUR else 'NOT '))



In [ ]:
def add_shifted_data(df):
    """
    Adds columns gps_wc{1/2/3}_INTIRX_m{15/30/45} to dataframe.
    Adds column  'datetime' if does not exists. Otherwise overwrites it.
    Changes atr_volume to present the cumulative count from the last hour (not from the last 15 minutes)
    """
    
    #df['datetime'] = df['datetime'] = pd.to_datetime(df.unix_ts,unit='s')#, utc='US/Eastern')   
    df['datetime'] = pd.to_datetime(df.date + ' ' + df.time)
    
    toShiftCols = ['gps_pt1_wc1', 'gps_pt2_wc1', 'gps_pt2_wc2', 'gps_pt2_wc3', 'volume']
    tmp = df[['tmc', 'datetime'] + toShiftCols].copy()
    for tshift in [15, 30, 45]:
        tmp2 = tmp.copy()
        cols = ['{}_m{}'.format(x, tshift) if x[-5:] == 'INRIX' or x=='volume' else x for x in list(tmp.columns)]
        tmp2.columns = cols
        tmp2.datetime = tmp2.datetime + pd.Timedelta(minutes=tshift)
        sys.stdout.write('Merging for {} minutes.  \r'.format(tshift))
        df = df.merge(tmp2, on=['tmc', 'datetime'], how='left')    


    df = df.dropna(subset=['volume_m{}'.format(x) for x in [15, 30, 45]])
    df = df[df.datetime >= '2018-01-02 01:00:00']
    for c in toShiftCols:
        df[c] = df[[c, '{}_m15'.format(c), '{}_m30'.format(c), '{}_m45'.format(c)]].sum(axis=1)
        df = df.drop(['{}_m15'.format(c), '{}_m30'.format(c), '{}_m45'.format(c)], axis=1)
    return df


In [ ]:
filename = './Example/results/DeepAndWide/20200418_results_all.csv'
file_id = '_'.join(filename[:-4].split('/')[-2:])

print (file_id)

sys.stdout.write('Loading data...\r')
results_all = pd.read_csv(filename)
original_shape = results_all.shape
results_all.rename(inplace=True, columns=
               {'count_location' : 'station', 
                'osm_highway': 'Road_type',
                'count_total' : 'volume',
                'pred' : 'pred_ANN',
               })


#print (results_all)
#results_all['pred_ANN'] = results_all['pred_tti'] # FOR TESTING TTI RESULTS


if CHANGE_TO_ONE_HOUR:
    results_all['pred_ANN_NotNull'] = ~results_all.pred_ANN.isnull()
    results_all = add_shifted_data(results_all)
    results = results_all[results_all['pred_ANN_NotNull']].copy().drop('pred_ANN_NotNull', axis='columns')
else:
    results = results_all[~pd.isnull(results_all['pred_ANN']) ].copy()


print ('Original results_all table had {} rows'.format(original_shape[0]))
print ('Results_all table has {} rows'.format(len(results_all)))
print ('Results table has {} rows ({:.2f} %)'.format(len(results), 100.0 * len(results) / len(results_all)))


colnames = ['gps_pt1_wc1', 'gps_pt2_wc1', 'gps_pt2_wc2', 'gps_pt2_wc3']
results[colnames] = results[colnames].fillna(0)
results['GPS'] = results[colnames].sum(axis=1)
print (results.shape)
print (results.columns)
results.head()

In [ ]:
#results.sort_values(['tmc', 'datetime_utc']).head()

### Compute errors 

In [ ]:
tmp = results.groupby('tmc').agg({'volume' : 'max'}).reset_index().rename(columns = {'volume' : 'capacityP'})
results = results.merge(tmp, on='tmc')

In [ ]:
#results['err_capacityT'] = np.abs(results['pred_ANN'] - results['volume']) / results['capacityT']
results['err_capacityP'] = 100 * np.abs(results['pred_ANN'] - results['volume']) / results['capacityP']
#results['mape'] = 100 * np.abs(results['pred_ANN'] - results['volume']) / results['volume']
#results['smape'] = 200 * np.abs(results.pred_ANN - results.volume) / ( results.pred_ANN.abs() + results.volume.abs() ) 
#results.head()

### Create list of parameters (locations, directions, road types)

In [ ]:
def create_list(rd_all, rd, listtype = 'tmc', verbose = 1):
    """
    Arguments:
        rd - dataframe with results
        listtype:
            'tmc' for stations
            'type' for road type
            'GPS' for number of GPS car observed
            'observations' - number of data points (rows)
    """
    stations = np.sort(pd.unique(rd['station']))
    ret = []
    if verbose > 0:
        sys.stdout.write('Generating list of parameters for {}.                  \r'.format(listtype))
    for station in stations:
        r_station = rd[rd['station'] == station]
        directions = np.sort(pd.unique(r_station['tmc']))
        for direction in directions:
            if listtype == 'station':
                ret.append(station)
            elif listtype == 'tmc':
                ret.append(direction)
            elif listtype == 'frc':
                rres = r_station[r_station['tmc'] == direction]
                ret.append(rres.iloc[0]['frc'])
            elif listtype == 'type':
                rres = r_station[r_station['tmc'] == direction]
                ret.append(rres.iloc[0]['Road_type'])
            elif listtype == 'lanes':
                rres = r_station[r_station['tmc'] == direction]
                ret.append(rres.iloc[0]['thru_lanes'])
            elif listtype == 'observations':
                rres = r_station[r_station['tmc'] == direction]
                ret.append(rres.shape[0])
            elif listtype == 'GPS':
                rres = r_station[r_station['tmc'] == direction]
                ret.append(int(np.sum(rres['GPS'])))
            elif listtype == 'volume':
                rres = r_station[r_station['tmc'] == direction]
                ret.append(int(np.sum(rres['volume'])))
            elif listtype == 'CapacityP':
                rres = r_station[r_station['tmc'] == direction]
                ret.append(int(np.max(rres['volume'])))                

    return ret
                
df_res = pd.DataFrame()
                
df_res['station'] = create_list(results_all, results, 'station')
df_res['tmc'] = create_list(results_all, results, 'tmc')
df_res['frc'] = create_list(results_all, results, 'frc')
df_res['type'] = create_list(results_all, results, 'type')
df_res['lanes'] = create_list(results_all, results, 'lanes')
df_res['observations'] = create_list(results_all, results, 'observations')
df_res['GPS'] = create_list(results_all, results, 'GPS')
df_res['volume'] = create_list(results_all, results, 'volume')
df_res['CapacityP'] = create_list(results_all, results, 'CapacityP')
df_res.head()

In [ ]:
df_res.describe()

### Compute and display $R^2$

In [ ]:
from sklearn.metrics import r2_score

def create_r2_table(rd):
    """
    Arguments:
        rd - DataFrame with results
    """
    
    res = pd.DataFrame(columns = ['station', 'tmc', 'r2'])
    stations = np.sort(pd.unique(rd['station']))
    i = 0
    ret = []
    for station in stations:
        r_station = rd[rd['station'] == station]
        directions = np.sort(pd.unique(r_station['tmc']))
        for direction in directions:
            rres = r_station[r_station['tmc'] == direction]
            r2 = r2_score(rres['volume'], rres['pred_ANN'])
            res.loc[i] = [station, direction, r2]
            i += 1
            #print (r2)
            ret.append(r2)
    return ret

df_res['R2'] = create_r2_table(results.copy())
df_res.head()

### Add errors to result table

In [ ]:
tmp = results.groupby('tmc').agg({
    'mape' : np.average,
    'smape' : np.average,
    'err_capacityP' : np.average,
    
}).rename(columns={
    'mape' : 'MAPE',
    'smape' : 'SMAPE',
    'err_capacityP' : 'EMFR'
})
df_res = df_res.merge(tmp, on='tmc')
df_res.head()

In [ ]:
def create_error_table(rd, column='volume'):
    """
    Arguments:
        column - may be 'capacityP', 'capacityP85', 'capacityT' or 'volume'  
    """
    
    err = pd.DataFrame(columns = ['station', 'tmc', 'err_ '+ column])
    stations = np.sort(pd.unique(rd['station']))
    i = 0
    ret = []
    for station in stations:
        r_station = rd[rd['station'] == station]
        directions = np.sort(pd.unique(r_station['tmc']))
        for direction in directions:
            rres = r_station[r_station['tmc'] == direction]
            e = np.average (rres['err_' + column] )
            err.loc[i] = [station, direction, e]
            i += 1
            #print (e)
            ret.append(e)
    return ret



# Old version, no need to use for MAPE, SMAPE and EMFR

#df_res['ECTR'] = create_error_table(results.copy(), 'capacityT') # ECR-T
#df_res['EMFR'] = create_error_table(results.copy(), 'capacityP') # ECR-P
#df_res.head()

In [ ]:
df_res.describe()

### Percentiles for GPS coutns

In [ ]:
df_res['GPS_avg'] = df_res.GPS / df_res.observations
perc_33 = np.percentile(df_res.GPS_avg, 33)
perc_67 = np.percentile(df_res.GPS_avg, 67)
print (perc_33, perc_67)
df_res['GPS_perc'] = 2
df_res.loc[df_res.GPS_avg <= perc_33, 'GPS_perc'] = 1
df_res.loc[df_res.GPS_avg >= perc_67, 'GPS_perc'] = 3
print ('Perc 33: {}, Perc 67: {}'.format(perc_33, perc_67))
print ('')
df_res.groupby('GPS_perc').agg({
    'R2' : [np.mean, np.median],
    'MAPE' : [np.mean, np.median],
    'EMFR' : [np.mean, np.median],
    'observations' : np.sum
    })


if CHANGE_TO_ONE_HOUR or (not GRAN_15_MIN): # 1 hour granulation
    gran_mul = 1
else:
    gran_mul = 4
    

print ('Granulation: {} minutes'.format(60 / gran_mul))    
for i in range (1, 4):
    print ('GPS_perc {}: {:.2f} - {:.2f}'
           .format(i, gran_mul * df_res.GPS_avg[df_res.GPS_perc == i].min(),
                   gran_mul * df_res.GPS_avg[df_res.GPS_perc == i].max()))



### Volume thresholds

In [ ]:
step = 1000
df_res['Vol_thresh'] = gran_mul * df_res.volume / df_res.observations / step
df_res.Vol_thresh = (df_res.Vol_thresh.astype(int) + 1) * step
df_res.loc[df_res.Vol_thresh >= 4000, 'Vol_thresh'] = 100000

### Day-Night and Peek Off-Peek
Day (6am-8pm) vs. night (8pm-6am)

peak (7am-9am, 4pm-6pm) vs. off-peak

In [ ]:
dfh = results.copy()
dfh['hour'] = pd.DatetimeIndex(dfh.datetime).hour  
dfh = dfh.set_index('datetime')
dfh.head()

Conditions

In [ ]:
daycond = (dfh.hour >= 6) & (dfh.hour < 20)
peekcond = ( (dfh.hour >= 7) & (dfh.hour < 9) ) | ( (dfh.hour >= 16) & (dfh.hour < 18) )

In [ ]:
from sklearn.metrics import r2_score

dnres = pd.DataFrame(columns=['observ.', 'R2', 'MAPE', 'SMAPE', 'EMFR'])
def add_to_res(dfh, dnres, desc, cond):
    dfhc = dfh[cond]
    r2 = r2_score(dfhc['volume'], dfhc['pred_ANN'])
    MAPE = np.average(dfhc['mape'])
    SMAPE = np.average(dfhc['smape'])
    #ETCR = np.average(dfhc['err_capacityT'])
    EMFR = np.average(dfhc['err_capacityP'])
    obs = dfhc.shape[0]
    dnres.loc[desc, :] = [obs, r2, MAPE, SMAPE, EMFR]
    
add_to_res(dfh, dnres, 'Day', daycond)   
add_to_res(dfh, dnres, 'Night', ~daycond)   
add_to_res(dfh, dnres, 'Peek', peekcond)
add_to_res(dfh, dnres, 'Off-Peek', ~peekcond)   

dnres.head()

# Detailed_results

In [ ]:
df_res.columns

In [ ]:
res_frc = df_res.groupby('frc').agg({
    'observations' : ['count', np.sum],
    'R2' : [np.mean, np.median],
    'MAPE' : [np.mean, np.median],
    'SMAPE' : [np.mean, np.median],
    'EMFR' : [np.mean, np.median],
})
res_frc

In [ ]:
res_gps = df_res.groupby('GPS_perc').agg({
    'observations' : ['count', np.sum],
    'R2' : [np.mean, np.median],
    'MAPE' : [np.mean, np.median],
    'SMAPE' : [np.mean, np.median],
    'EMFR' : [np.mean, np.median],
})
res_gps

In [ ]:
print ('Granulation: {} minutes'.format(60 / gran_mul))    
for i in range (1, 4):
    print ('GPS_perc {}: {:.2f} - {:.2f}'
           .format(i, gran_mul * df_res.GPS_avg[df_res.GPS_perc == i].min(), 
                   gran_mul * df_res.GPS_avg[df_res.GPS_perc == i].max()))



In [ ]:
res_vol = df_res.groupby('Vol_thresh').agg({
    'observations' : ['count', np.sum],
    'R2' : [np.mean, np.median],
    'MAPE' : [np.mean, np.median],
    'SMAPE' : [np.mean, np.median],    
    'EMFR' : [np.mean, np.median],
})

res_vol

# Charts

In [ ]:
from math import log10, floor
def round_sig(x, sig=2):
    res = round(x, sig-int(floor(log10(abs(x))))-1)
    return int(res) if res == int(res) else res
        

    
def plot_bar(df, metric = 'R2', metric_agg = 'median', 
             title = None, x_labels = None, ymax = None, 
             file_pref = None, file_format = 'jpg'):
    
    if x_labels is None:
        x_labels = df.index
        
    pos = np.arange(len(x_labels))
    x_val = df[metric][metric_agg]

    plt.figure()

    bars = plt.bar(pos, x_val, align='center',  )

    # Add title
    if title is not None:
        title = '{} ${}$ for {}'.format(
            metric_agg.capitalize(), 
            ('R^2' if metric == 'R2' else '{} [\%]'.format(metric)),
            title
        )
        plt.title(title, alpha=0.8)
        
    plt.xticks(pos, x_labels, alpha=0.8)
    if ymax is not None:
        plt.ylim([0, ymax])
    
    #Remove ticks and labels
    plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=True)

    #remove frames
    for spine in plt.gca().spines.values():
        spine.set_visible(False)

    height_shift = x_val.max() * 0.1
    # direct label each bar with Y axis values
    for bar in bars:
        disp_val = round_sig(bar.get_height(), 2)
        plt.gca().text(bar.get_x() + bar.get_width()/2, bar.get_height() - height_shift, disp_val, 
                     ha='center', color='w', fontsize=20)
    
    if file_pref is not None:
        # Note = file_id is a global variable
        filename = '{}_{}_{}_{}.{}'.format(file_pref, metric, metric_agg, file_id, file_format)
        plt.savefig(filename)


In [ ]:
metrics = ['EMFR', 'R2', 'MAPE', 'SMAPE']
ymax_list = [None] * len(metrics)
ymax_list = [10, 0.92, 38, 36]

In [ ]:
CHART_DIR = "./MetricImgs"
import os
if not os.path.exists(CHART_DIR):
    os.makedirs(CHART_DIR)

In [ ]:
for m, ymax in zip(metrics, ymax_list):
    plot_bar(res_gps, metric=m, 
             title = 'different avg probe counts / h', x_labels = ['Low', 'Med', 'High'], ymax = ymax,
            file_pref = '{}/res_gps'.format(CHART_DIR))

In [ ]:
for m, ymax in zip(metrics, ymax_list):
    plot_bar(res_frc, metric=m, 
             title = 'different FRCs', x_labels = ['Frc 1', 'Frc 2', 'Frc 3+'], ymax = ymax,
            file_pref = '{}/res_frc'.format(CHART_DIR))

In [ ]:
for m, ymax in zip(metrics, ymax_list):
    plot_bar(res_vol, metric=m, 
             title = 'different avg hourly volume thresholds', x_labels = ['0-1k', '1k-2k', '2k-3k', '3k+'], ymax = ymax,
            file_pref = '{}/res_vol'.format(CHART_DIR))